#  FOR GATK (scATAC-seq)

FASTQ to bam before call variants

In [ ]:
!gunzip *.fastq.gunzip

In [ ]:
!bwa mem -t 8 ~/hg38.fa \
    sample1_1.fastq \
    sample1_2.fastq > aligned_sample.sam

Get cell barcode and add CB information to bam

In [ ]:
!awk 'NR%4==1 {split($2, a, ":"); print substr($1,2), a[1]}' sample1_1.fastq sample1_2.fastq > all_barcode_list.txt

In [ ]:
!awk 'BEGIN {OFS="\t"}
     NR==FNR {barcode[$1]=$2; next} 
     $1 in barcode {print $0, "CB:Z:"barcode[$1]}  
     !($1 in barcode) {print $0}' all_barcode_list.txt aligned_sample.sam > output_with_barcode.sam


Sort and Index

In [ ]:
!samtools view -bS output_with_barcode.sam | samtools sort -o output_with_barcode_sorted.bam

!samtools index output_with_barcode_sorted.bam

Add read group information (if not provided )

In [ ]:
!java -jar ~/picard.jar AddOrReplaceReadGroups \
    I=~/output_with_barcode_sorted.bam \
    O=~/sample.sort.rg.bam \
    RGID=tissue RGLB=lib_all RGPL=illumina RGPU=sn RGSM=SN-A8WNZ

### If a suitable bam file has been obtained

### The split_cell.sh file can be used directly to subset the single-cell dataset by cell type.

Just modify the input and output files to use 

eg:run sbatch split_cell.sh in terminal

In [ ]:
#!/bin/bash

#SBATCH --nodes=1              
#SBATCH --ntasks-per-node=10    

# Define input and output files
input_bam="~sample.sort.rg.bam"
input_dir="~/celltype1/"
output_dir="~/celltype1/ori_bam/"
barcodes_file="~/celltype1_barcode.txt"
chromosomes_file="~/standard_chromosomes.txt"

# Filter low-quality reads and remove duplicates
filtered_bam="${input_dir}/filtered.bam"
result_bam="${input_dir}/result.bam"
samtools view -b -q 30 $input_bam > $filtered_bam
#only keep standard chromosomes
samtools view -h $filtered_bam | awk 'NR==FNR{{a[$1]; next}} $1 ~ /^@/ || $3 in a' $chromosomes_file - | samtools view -b -o $result_bam

# Index the filtered BAM file
samtools index $result_bam

# Split BAM file by barcode using samtools and awk
while read barcode; do
    output_bam="${output_dir}/${barcode}_cell.bam"
    samtools view -h $result_bam | awk -v barcode="CB:Z:$barcode" '$0 ~ barcode || $1 ~ /^@/' | samtools view -b -o $output_bam

    samtools index $output_bam
done < $barcodes_file

# Verify the number of unique barcodes
wc -l $barcodes_file


# CALL SNV

In [ ]:
#call snp
import subprocess
import os



def call_snv(bam_file, ref, output_dir, gatk_path, dbsnp):
    vcf_output = os.path.join(output_dir, bam_file.split('/')[-1].replace('.bam', '.vcf'))
    cmd = [
        gatk_path, "HaplotypeCaller",
        "-R", ref,
        "-I", bam_file,
        "--dbsnp", dbsnp,
        "-O", vcf_output
    ]
    subprocess.run(cmd, check=True)

filtered_bam_folder = "~/celltype1/ori_bam/"
filtered_bam_files = [f for f in os.listdir(filtered_bam_folder) if f.endswith('.bam')]
filtered_bam_files = [os.path.join(filtered_bam_folder, f) for f in filtered_bam_files]

for bam_file in filtered_bam_files:
    # Call SNV on the filtered BAM file
    call_snv(bam_file,
             "~/hg38.fa",
             "~/snp_out/",
             "gatk",
             "~/resources_broad_hg38_v0_Homo_sapiens_assembly38.dbsnp138.vcf")

